In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

Pandas Settings:

In [3]:
pd.set_option("max_colwidth", 150)
pd.set_option("display.max_columns", 60)
pd.set_option('max_rows', None)
# pd.set_option("max_colwidth", 250)

Okay heres the order for the MVP training set:
1. Drop non-MVP columns
1. Drop NaN values
1. OHC Sex column
1. Split X and Y
1. Fit
1. Score

## Importing Datasets

In [18]:
df_train = pd.read_csv(r'C:\Users\ggomp\OneDrive\Documents\GitHub\Kaggle_Competitions\titanic_classification\original_data\train.csv')
df_test = pd.read_csv(r'C:\Users\ggomp\OneDrive\Documents\GitHub\Kaggle_Competitions\titanic_classification\original_data\test.csv')
df_sub = pd.read_csv(r'C:\Users\ggomp\OneDrive\Documents\GitHub\Kaggle_Competitions\titanic_classification\original_data\gender_submission.csv')

display(df_train.head(2))
display(df_test.tail(2))
display(df_sub.head(2))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


,PassengerId,Survived
0,892,0
1,893,1


1. Drop non-MVP columns from both train and test sets

In [19]:
train_copy = df_train.copy()
train_mvp = train_copy.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
display(train_mvp.head(2))

test_copy = df_test.copy()
test_mvp = test_copy.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
display(test_mvp.tail(2))

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
416,3,male,NaN,0,0,8.0500,S
417,3,male,NaN,1,1,22.3583,C


## Checking for NaN values

In [6]:
# print("df_train:\n", df_train.isnull().sum())
print('train set:')
print(train_mvp.isnull().sum())

print()
print('test set:')
print(test_mvp.isnull().sum())

# print(train_mvp['Embarked'].isnull())

train set:
Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

test set:
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


## OHC 2 Columns

In [20]:
#train set
print('train set')
train_mvp_c = train_mvp.copy()
train_mvp_c = pd.get_dummies(train_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
train_mvp_c = pd.get_dummies(train_mvp_c,prefix=['Embarked'], columns = ['Embarked'], drop_first=True)

display(train_mvp_c.head(2))
print(train_mvp_c.isnull().sum())
print('\n\n')
# display(train_mvp_c.iloc[59:64])

# test set
print('test set')
test_mvp_c = test_mvp.copy()
test_mvp_c = pd.get_dummies(test_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
test_mvp_c = pd.get_dummies(test_mvp_c,prefix=['Embarked'], columns = ['Embarked'], drop_first=True)

display(test_mvp_c.tail(2))
print(test_mvp_c.isnull().sum())


train set


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0


Survived        0
Pclass          0
Age           177
SibSp           0
Parch           0
Fare            0
Sex_male        0
Embarked_Q      0
Embarked_S      0
dtype: int64



test set


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
416,3,NaN,0,0,8.0500,1,0,1
417,3,NaN,1,1,22.3583,1,0,0


Pclass         0
Age           86
SibSp          0
Parch          0
Fare           1
Sex_male       0
Embarked_Q     0
Embarked_S     0
dtype: int64


## Imputting NaN values

In [21]:
imputer = IterativeImputer()
# imputer = KNNImputer(n_neighbors=8)

# Train Set
print("Train Set")
train_cols = train_mvp_c.columns
train_mvp_c = imputer.fit_transform(train_mvp_c)
train_mvp_c = pd.DataFrame(train_mvp_c, columns=train_cols)
display(train_mvp_c.tail(3))
print(f'{train_mvp_c.isnull().sum()}\n')

#Test Set
print('Test Set')
test_mvp_c = imputer.fit_transform(test_mvp_c)
test_mvp_c = pd.DataFrame(test_mvp_c)
display(test_mvp_c.tail(3))
print(test_mvp_c.isnull().sum())

Train Set
 these are the train_cols: Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
888,0.0,3.0,23.190155,1.0,2.0,23.45,0.0,0.0,1.0
889,1.0,1.0,26.000000,0.0,0.0,30.00,1.0,0.0,0.0
890,0.0,3.0,32.000000,0.0,0.0,7.75,1.0,1.0,0.0


Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_male      0
Embarked_Q    0
Embarked_S    0
dtype: int64



Test Set


,0,1,2,3,4,5,6,7
415,3.0,38.500000,0.0,0.0,7.2500,1.0,0.0,1.0
416,3.0,24.941183,0.0,0.0,8.0500,1.0,0.0,1.0
417,3.0,23.236983,1.0,1.0,22.3583,1.0,0.0,0.0


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64


2. Drop NaN values


In [22]:
train_mvp_c = train_mvp_c.dropna(axis = 0)
train_mvp_c = train_mvp_c.reset_index(drop=True)
display(train_mvp_c.tail(5))

test_mvp_c = test_mvp_c.dropna(axis = 0)
test_mvp_c = test_mvp_c.reset_index(drop=True)
display(test_mvp_c.tail(5))

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
886,0.0,2.0,27.000000,0.0,0.0,13.00,1.0,0.0,1.0
887,1.0,1.0,19.000000,0.0,0.0,30.00,0.0,0.0,1.0
888,0.0,3.0,23.190155,1.0,2.0,23.45,0.0,0.0,1.0
889,1.0,1.0,26.000000,0.0,0.0,30.00,1.0,0.0,0.0
890,0.0,3.0,32.000000,0.0,0.0,7.75,1.0,1.0,0.0


,0,1,2,3,4,5,6,7
413,3.0,24.941183,0.0,0.0,8.0500,1.0,0.0,1.0
414,1.0,39.000000,0.0,0.0,108.9000,0.0,0.0,0.0
415,3.0,38.500000,0.0,0.0,7.2500,1.0,0.0,1.0
416,3.0,24.941183,0.0,0.0,8.0500,1.0,0.0,1.0
417,3.0,23.236983,1.0,1.0,22.3583,1.0,0.0,0.0


3. Split X and Y

In [23]:
# Train Set
x_train, y_train = train_mvp_c.iloc[:,1:], train_mvp_c.iloc[:, 0]
display(x_train.tail(2))
display(y_train.tail(2))

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
889,1.0,26.0,0.0,0.0,30.00,1.0,0.0,0.0
890,3.0,32.0,0.0,0.0,7.75,1.0,1.0,0.0


889    1.0
890    0.0
Name: Survived, dtype: float64

## Exporting proccesed data to CSV

In [24]:
x_train.to_csv(r'.\proccesed_data\x_train.csv', index=False)
y_train.to_csv(r'.\proccesed_data\y_train.csv', index=False)
test_mvp_c.to_csv(r'.\proccesed_data\x_test.csv', index=False)

## Continue to Training Notebook